## Lesson 6: Planning and Stock Report Generation

https://learn.deeplearning.ai/courses/ai-agentic-design-patterns-with-autogen/lesson/7/planning-and-stock-report-generation

In [1]:
from autogen import AssistantAgent, ConversableAgent, GroupChat, GroupChatManager
from dotenv import find_dotenv, load_dotenv

In [2]:
load_dotenv(find_dotenv())

MODEL_NAME = "glm-4"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

llm_config = {"model": MODEL_NAME, "base_url": BASE_URL, "temperature": 0.5}

### The task!

In [3]:
task = """
    Write a blogpost about the stock price performance of
    NVIDIA in the past month. Today's date is 2024-06-10.
"""

### Build a group chat
This group chat will include these agents:
- User_proxy or Admin: to allow the user to comment on the report and ask the writer to refine it
- Planner: to determine relevant information needed to complete the task
- Engineer: to write code using the defined plan by the planner
- Executor: to execute the code written by the engineer
- Writer: to write the report

In [4]:
user_proxy = ConversableAgent(
    name="Admin",
    system_message="Give the task, and send instructions to writer to refine the blogpost.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

executor = ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the " "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

writer = ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog.",
)

### Define the group chat

In [5]:
groupchat = GroupChat(
    agents=[user_proxy, planner, engineer, executor, writer],
    messages=[],
    max_round=10,
)

manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

### Start the group chat!

In [ ]:
groupchat_result = user_proxy.initiate_chat(recipient=manager, message=task)

### Add a speaker selection policy

In [7]:
user_proxy = ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan " "provided by the planner.",
)

writer = ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = ConversableAgent(
    name="Executor",
    description="Execute the code written by the " "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [8]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [ ]:
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

groupchat_result = user_proxy.initiate_chat(
    recipient=manager,
    message=task,
)